# Synaptipy — Batch Analysis Example

This notebook demonstrates how to use Synaptipy’s core analysis engine
programmatically — without opening the GUI — to:

1. Load electrophysiology files from disk (ABF format)
2. Run spike detection and input-resistance analysis
3. Collect results into a Pandas DataFrame
4. Plot summary statistics

**Requirements:** `pip install synaptipy matplotlib pandas`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Synaptipy core imports
from Synaptipy.infrastructure.file_readers.neo_adapter import NeoAdapter
from Synaptipy.core.analysis.spike_analysis import detect_spikes_threshold
from Synaptipy.core.analysis.intrinsic_properties import calculate_rin
from Synaptipy.core.analysis.basic_features import calculate_rmp

print("Synaptipy imported successfully.")

## 1. Discover Data Files

Point to the `examples/data/` folder that ships with the repository.
These are Axon ABF recordings from whole-cell patch-clamp experiments.

In [ ]:
data_dir = Path("data")
abf_files = sorted(data_dir.glob("*.abf"))
print(f"Found {len(abf_files)} ABF files:")
for f in abf_files:
    print(f"  {f.name}")

## 2. Load and Inspect a Single Recording

Use `NeoAdapter` to read the file into Synaptipy’s `Recording` data model.

In [ ]:
adapter = NeoAdapter()
recording = adapter.read_recording(abf_files[0])

print(f"File: {abf_files[0].name}")
print(f"Sampling rate: {recording.sampling_rate} Hz")
print(f"Channels: {len(recording.channels)}")
for ch_id, ch in recording.channels.items():
    n_trials = len(ch.data_trials) if ch.data_trials else 0
    print(f"  [{ch_id}] {ch.name} ({ch.units}) — {n_trials} trial(s)")

## 3. Analyse a Single Trace

Extract the first voltage channel and run RMP + spike detection.

In [ ]:
# Find the first voltage channel
v_channel = None
for ch in recording.channels.values():
    if ch.units and ch.units.lower() in ("mv", "v"):
        v_channel = ch
        break

if v_channel is None:
    print("No voltage channel found — using first channel.")
    v_channel = list(recording.channels.values())[0]

data = v_channel.data_trials[0]  # first trial
fs = v_channel.sampling_rate or recording.sampling_rate
dt = 1.0 / fs
time = np.arange(len(data)) * dt + float(getattr(v_channel, 't_start', 0.0))

print(f"Channel: {v_channel.name}, {len(data)} samples, {fs} Hz")
print(f"Duration: {time[-1] - time[0]:.3f} s")

In [ ]:
# Plot the raw trace
fig, ax = plt.subplots(figsize=(12, 3))
ax.plot(time, data, linewidth=0.5)
ax.set_xlabel("Time (s)")
ax.set_ylabel(f"Voltage ({v_channel.units})")
ax.set_title(f"{abf_files[0].name} — Trial 1")
plt.tight_layout()
plt.show()

In [ ]:
# Spike detection
refractory_samples = int(0.002 * fs)  # 2 ms refractory period
spike_result = detect_spikes_threshold(
    data, time,
    threshold=-20.0,
    refractory_samples=refractory_samples,
)

n_spikes = len(spike_result.spike_times) if spike_result.spike_times is not None else 0
print(f"Spikes detected: {n_spikes}")
if n_spikes > 0:
    print(f"Mean frequency: {spike_result.mean_frequency:.1f} Hz")
    print(f"Spike times (first 5): {spike_result.spike_times[:5]}")

In [ ]:
# RMP analysis
rmp_result = calculate_rmp(
    data, time,
    baseline_window=(time[0], time[0] + 0.1),
)
print(f"RMP: {rmp_result.value:.2f} {rmp_result.unit}")

## 4. Batch Processing Across All Files

Loop over every ABF file, run spike detection, and collect results
into a Pandas DataFrame.

In [ ]:
rows = []

for fpath in abf_files:
    try:
        rec = adapter.read_recording(fpath)
    except Exception as e:
        print(f"  Skipping {fpath.name}: {e}")
        continue

    # Find voltage channel
    v_ch = None
    for ch in rec.channels.values():
        if ch.units and ch.units.lower() in ("mv", "v"):
            v_ch = ch
            break
    if v_ch is None:
        v_ch = list(rec.channels.values())[0]

    fs_file = v_ch.sampling_rate or rec.sampling_rate
    refractory = int(0.002 * fs_file)

    for trial_idx, trial_data in enumerate(v_ch.data_trials):
        t = np.arange(len(trial_data)) / fs_file
        sr = detect_spikes_threshold(
            trial_data, t,
            threshold=-20.0,
            refractory_samples=refractory,
        )
        n_sp = len(sr.spike_times) if sr.spike_times is not None else 0
        rows.append({
            "file": fpath.name,
            "channel": v_ch.name,
            "trial": trial_idx,
            "spike_count": n_sp,
            "mean_freq_hz": sr.mean_frequency if n_sp > 0 else 0.0,
        })

df = pd.DataFrame(rows)
print(f"Processed {len(abf_files)} files, {len(df)} total sweeps.")
df.head(10)

In [ ]:
# Summary statistics per file
summary = df.groupby("file").agg(
    total_spikes=("spike_count", "sum"),
    mean_freq=("mean_freq_hz", "mean"),
    n_trials=("trial", "count"),
).reset_index()

print(summary.to_string(index=False))

In [ ]:
# Bar chart of spike counts per file
fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(summary["file"], summary["total_spikes"])
ax.set_ylabel("Total Spike Count")
ax.set_xlabel("File")
ax.set_title("Spike Counts Across Recordings")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

## 5. Export Results to CSV

Save the full results DataFrame for downstream analysis in R, MATLAB, or
any other statistics package.

In [ ]:
output_csv = Path("batch_spike_results.csv")
df.to_csv(output_csv, index=False)
print(f"Results saved to {output_csv}")